In [8]:
import pandas as pd

df1 = pd.read_csv('/Users/IMYV/Desktop/cfdb-django/data/tablets.csv', usecols=['title', 'region', 'period', 'Year'], engine='python')
df2 = pd.read_csv('/Users/IMYV/Desktop/cfdb-django/data/glyphs.csv', usecols=['tablet', 'sign', 'image'], engine='python')
df3 = pd.read_csv('/Users/IMYV/Desktop/cfdb-django/data/stdSigns.csv', usecols=['sign_name', 'image'], engine='python')

df4 = df1.merge(df2, left_on='title', right_on='tablet', how='right')
df5 = df4.drop(['tablet'], axis=1)

df6 = df5.merge(df3, left_on='sign', right_on='sign_name', how='right')
df7 = df6.drop(['sign_name'], axis=1)
df8 = df7.rename(index=str, columns={"title": "tablet", "Year": "year", "image_x": "glyph", "image_y": "std_sign"})

print  df8.duplicated(subset=, keep='first')[source]

,tablet,region,period,year,sign,glyph,std_sign
0,BM 94594,Borsippa,nb,583.0,SI,7868344e-1a0f-4291-821d-501d9b7d753e.png,sign_112.bmp
1,NCBT 524,Uruk,achaemenid,528.0,SI,9fe958e1-043e-4271-a089-c42e36dc7239.png,sign_112.bmp
2,NBC 6126,Nippur,achaemenid,533.0,SI,6a6d28ea-89b2-4fb8-b5dc-dd397d5aac30.png,sign_112.bmp
3,JCS 36 8,Uruk,enb,642.0,SI,3ddff6f7-c71b-4eb7-a63d-7d06fba29287.png,sign_112.bmp
4,NBC 6133,Nippur,achaemenid,533.0,SI,1ac15ca0-39b7-4fe1-bde5-0e0c8a8e8b8b.png,sign_112.bmp
5,BM 79103,Sippar,achaemenid,499.0,SI,e94f128b-2a0a-4394-9edc-2a78d6785f8f.png,sign_112.bmp
6,BCHP 14,Babylon,peri_seleukid,NaN,SI,1cf4545a-5c91-4a17-afef-357bde5f4779.png,sign_112.bmp
7,BM 42356+,Sippar,achaemenid,492.0,SI,59939fb3-c493-4182-a13a-fbfa47477a42.png,sign_112.bmp
8,BCHP 11,Babylon,peri_seleukid,NaN,SI,a9d5f6e8-8648-48d7-b772-41b605f6f208.png,sign_112.bmp
9,BM 64240,Sippar,achaemenid,512.0,SI,193dbe32-e2d9-488a-bb4e-4ac14ae58bd4.png,sign_112.bmp
